In [1]:
import tensorflow as tf

import numpy as np
import os
import time

In [2]:
path_to_file = 'wp.txt'

In [9]:
text = open(path_to_file, 'rb').read().decode(encoding='utf-8', errors="ignore")
# length of text is the number of characters in it
print('Length of text: {} characters'.format(len(text)))

Length of text: 1551216 characters


In [10]:
print(text[:500])

﻿
Замысел

В 1855 году появилось объявление об издании «Полярной звезды». На обложке книги в круге восходящего солнца были изображены пять портретов казненных декабристов; под портретами топор и подписано: «25 июля 1826 года». Том помечен днем казни декабристов.

Над заглавием в тучах звезда.

Полярная.

Объявление было целым манифестом. Герцен говорил о восстании декабристов и о севастопольской кампании; спрашивал, неужели «севастопольский солдат, израненный и твердый, как гранит, испы


In [11]:
text = text + text

In [12]:
# The unique characters in the file
vocab = sorted(set(text))
print('{} unique characters'.format(len(vocab)))

154 unique characters


In [13]:
# Creating a mapping from unique characters to indices
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

text_as_int = np.array([char2idx[c] for c in text])

text_as_int, text, len(text_as_int), len(text)

### train and target

In [15]:
# The maximum length sentence you want for a single input in characters
seq_length = 100
examples_per_epoch = len(text)//(seq_length+1)

# Create training examples / targets
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

for i in char_dataset.take(5):
    print(idx2char[i.numpy()])

﻿



З
а


In [16]:
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

for item in sequences.take(5):
    print(repr(''.join(idx2char[item.numpy()])))

'\ufeff\r\nЗамысел\r\n\r\nВ 1855 году появилось объявление об издании «Полярной звезды». На обложке книги в круге'
' восходящего солнца были изображены пять портретов казненных декабристов; под портретами топор и подп'
'исано: «25 июля 1826 года». Том помечен днем казни декабристов.\r\n\r\nНад заглавием в тучах звезда.\r\n\r\nП'
'олярная.\r\n\r\nОбъявление было целым манифестом. Герцен говорил о восстании декабристов и о севастопольс'
'кой кампании; спрашивал, неужели «севастопольский солдат, израненный и твердый, как гранит, испытавши'


In [17]:
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)

Print the first example input and target values:

In [18]:
for input_example, target_example in  dataset.take(1):
    print('Input data: ', repr(''.join(idx2char[input_example.numpy()])))
    print('Target data:', repr(''.join(idx2char[target_example.numpy()])))

Input data:  '\ufeff\r\nЗамысел\r\n\r\nВ 1855 году появилось объявление об издании «Полярной звезды». На обложке книги в круг'
Target data: '\r\nЗамысел\r\n\r\nВ 1855 году появилось объявление об издании «Полярной звезды». На обложке книги в круге'


In [19]:
# Batch size
BATCH_SIZE = 64

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

dataset

<BatchDataset element_spec=(TensorSpec(shape=(64, 100), dtype=tf.int32, name=None), TensorSpec(shape=(64, 100), dtype=tf.int32, name=None))>

In [20]:
# Length of the vocabulary in chars
vocab_size = len(vocab)

# The embedding dimension
embedding_dim = 128

# Number of RNN units
rnn_units = 1024

In [46]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
    model = tf.keras.Sequential([
        tf.keras.layers.Embedding(vocab_size, embedding_dim,
                                  batch_input_shape=[batch_size, None]),
                                 
        tf.keras.layers.LSTM(rnn_units,
                            return_sequences=True,
                            stateful=True,
                            recurrent_initializer='glorot_uniform'),
        tf.keras.layers.LSTM(rnn_units,
                            return_sequences=True,
                            stateful=True,
                            recurrent_initializer='glorot_uniform'),
        tf.keras.layers.LSTM(rnn_units,
                            return_sequences=True,
                            stateful=True,
                            recurrent_initializer='glorot_uniform'),

        tf.keras.layers.LSTM(rnn_units,
                            return_sequences=True,
                            stateful=True,
                            recurrent_initializer='glorot_uniform'),                                   
        tf.keras.layers.Dense(512, activation='relu'),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(vocab_size)
        
    ])
    return model

In [47]:
model = build_model(
    vocab_size=len(vocab),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units,
    batch_size=BATCH_SIZE)

In [48]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 100, 154) # (batch_size, sequence_length, vocab_size)


In [49]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_3 (Embedding)     (64, None, 128)           19712     
                                                                 
 lstm_6 (LSTM)               (64, None, 1024)          4722688   
                                                                 
 lstm_7 (LSTM)               (64, None, 1024)          8392704   
                                                                 
 lstm_8 (LSTM)               (64, None, 1024)          8392704   
                                                                 
 lstm_9 (LSTM)               (64, None, 1024)          8392704   
                                                                 
 dense_6 (Dense)             (64, None, 512)           524800    
                                                                 
 dropout_3 (Dropout)         (64, None, 512)          

In [50]:
example_batch_predictions[0]

<tf.Tensor: shape=(100, 154), dtype=float32, numpy=
array([[-1.1064605e-05, -4.1784342e-06, -3.2332746e-06, ...,
         9.3372037e-06,  4.1269832e-06,  1.8973991e-05],
       [-3.7629998e-05, -1.9898778e-05, -1.4448193e-05, ...,
         1.8569746e-05,  8.7193830e-06,  6.2103019e-05],
       [-7.5330885e-05, -5.6368321e-05, -3.6117464e-05, ...,
         2.8431534e-05,  1.3127819e-05,  1.0866790e-04],
       ...,
       [-1.4772121e-04, -3.7256964e-03, -5.0959870e-04, ...,
         1.1404677e-03, -1.3160257e-03,  4.3925614e-04],
       [-1.0278530e-04, -3.8756812e-03, -4.9092807e-04, ...,
         1.1077581e-03, -1.3417978e-03,  4.5031475e-04],
       [-4.8906077e-05, -3.9888937e-03, -4.8147270e-04, ...,
         1.0631204e-03, -1.3568229e-03,  5.0633756e-04]], dtype=float32)>

In [51]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices,axis=-1).numpy()

In [52]:
print("Input: \n", repr("".join(idx2char[input_example_batch[0]])))
print()
print("Next Char Predictions: \n", repr("".join(idx2char[sampled_indices ])))

Input: 
 ' было радостно и счастливо. Получив приказание, он пустил лошадь и поскакал вдоль по линии. Сначала '

Next Char Predictions: 
 'cэUDuNoB„](fF(зiC—[zQ3fсP…beЙsqАZп]hF0КO\xa0еНIЩ&ФЖШ’М.(рFкTЮO9 жЕДпRu\ufeffxдwхЭXаCиCaяhGuMДta?9n6оGЙЗЛрПPE'


## Train the model

In [53]:
def loss(labels, logits):
    return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

example_batch_loss = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("scalar_loss:      ", example_batch_loss.numpy().mean())

Prediction shape:  (64, 100, 154)  # (batch_size, sequence_length, vocab_size)
scalar_loss:       5.036977


In [54]:
model.compile(optimizer='adam', loss=loss)

### Configure checkpoints

In [55]:
!rm -rf ./training_checkpoints

"rm" ­Ґ пў«пҐвбп ў­гваҐ­­Ґ© Ё«Ё ў­Ґи­Ґ©
Є®¬ ­¤®©, ЁбЇ®«­пҐ¬®© Їа®Ја ¬¬®© Ё«Ё Ї ЄҐв­л¬ д ©«®¬.


In [56]:
!ls ./training_checkpoints

"ls" ­Ґ пў«пҐвбп ў­гваҐ­­Ґ© Ё«Ё ў­Ґи­Ґ©
Є®¬ ­¤®©, ЁбЇ®«­пҐ¬®© Їа®Ја ¬¬®© Ё«Ё Ї ЄҐв­л¬ д ©«®¬.


In [57]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_freq=88*3,
    save_weights_only=True)

### Execute the training

In [58]:
EPOCHS = 50

In [59]:
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/50
479/479 [==============================] - 91s 180ms/step - loss: 2.7082
Epoch 2/50
479/479 [==============================] - 86s 179ms/step - loss: 1.7829
Epoch 3/50
479/479 [==============================] - 87s 180ms/step - loss: 1.4953
Epoch 4/50
479/479 [==============================] - 86s 178ms/step - loss: 1.3791
Epoch 5/50
479/479 [==============================] - 86s 179ms/step - loss: 1.2988
Epoch 6/50
479/479 [==============================] - 85s 176ms/step - loss: 1.2291
Epoch 7/50
479/479 [==============================] - 86s 178ms/step - loss: 1.1625
Epoch 8/50
479/479 [==============================] - 87s 180ms/step - loss: 1.0958
Epoch 9/50
479/479 [==============================] - 86s 178ms/step - loss: 1.0291
Epoch 10/50
479/479 [==============================] - 87s 179ms/step - loss: 0.9620
Epoch 11/50
479/479 [==============================] - 85s 176ms/step - loss: 0.8947
Epoch 12/50
479/479 [==============================] - 86s 178ms/step - lo

## Generate text

In [61]:
tf.train.latest_checkpoint(checkpoint_dir)

'./training_checkpoints\\ckpt_50'

In [62]:
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)
model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))
model.build(tf.TensorShape([1, None]))

In [63]:
model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_4 (Embedding)     (1, None, 128)            19712     
                                                                 
 lstm_10 (LSTM)              (1, None, 1024)           4722688   
                                                                 
 lstm_11 (LSTM)              (1, None, 1024)           8392704   
                                                                 
 lstm_12 (LSTM)              (1, None, 1024)           8392704   
                                                                 
 lstm_13 (LSTM)              (1, None, 1024)           8392704   
                                                                 
 dense_8 (Dense)             (1, None, 512)            524800    
                                                                 
 dropout_4 (Dropout)         (1, None, 512)           

In [64]:
def generate_text(model, start_string):
    # Evaluation step (generating text using the learned model)

    # Number of characters to generate
    num_generate = 500

    # Converting our start string to numbers (vectorizing)
    input_eval = [char2idx[s] for s in start_string]
    input_eval = tf.expand_dims(input_eval, 0)

    # Empty string to store our results
    text_generated = []

    # Low temperature results in more predictable text.
    # Higher temperature results in more surprising text.
    # Experiment to find the best setting.
    temperature = 0.5

    # Here batch size == 1
    model.reset_states()
    for i in range(num_generate):
        predictions = model(input_eval)
        predictions = tf.squeeze(predictions, 0)
        # using a categorical distribution to predict the character returned by the model
        predictions = predictions / temperature
        predicted_id = tf.random.categorical(predictions, num_samples=1)[-1, 0].numpy()

        # Pass the predicted character as the next input to the model
        # along with the previous hidden state
        input_eval = tf.expand_dims([predicted_id], 0)

        text_generated.append(idx2char[predicted_id])

    return (start_string + ''.join(text_generated))

In [65]:
text_ = generate_text(model, start_string=u"Что ты знаешь о русской душе ")
print(text_)

Что ты знаешь о русской душе спокойно покинуть эту…

— Да, это славно, — закричал он так, что жилы, когда камердировканный солдат с крыго на него, и с неестественной улыбкой сказал Пьер. — Очень вам благодарен… Вы откуда изволите проезжать? — Лицо проезжающего было не ласково, даже холоднее и спокойнее.


В пять часов утра еще было совсем темно. Войска центра, резервов и правый фланг Багратиона с известил о своем приезде, никуда не выезжал и стал целые дни проводить за чтение. Кутузов остановился, разговаривая с австри


In [66]:
len(text_)

529